# Demo UI

# Table Schema

```
CREATE TABLE IF NOT EXISTS workspan.customer_opportunities (
    customer_id text,
    partner_id text,
    opportunity_id text,
    customer_name text static,
    next_step text,
    cadence text,
    llm_output text,
    opportunity map<text, text>,
    llm_output_embedding vector<float, 1536>,
    sentiment text,
    PRIMARY KEY ((customer_id, partner_id), opportunity_id)
) WITH CLUSTERING ORDER BY (opportunity_id DESC)

```



* customer_id: Unique identifier for each customer
* partner_id: AWS / Azure / GCP
* opportunity_id: Unique identifier for each opportunity
* opportunity: Dynamic collection of data field names and corresponding values specific to the opportunity. Other fields can be stored in separate table columns.
* llm_ouput: LLM output summarizing the 'next steps' , 'challenges' and 'open items'  
* llm_ouput_embedding: Text embeddings corresponding to llm_output
* sentiment: Positive, Neutral and Negative sentiment derived from 'next step' and 'cadence' data fields. This can be determined either by using a sentiment analysis library such as  Python Natural Language Toolkit (NLTK) or from LLM.


# Imports

In [ ]:
!pip install openai cassandra-driver llama-index gradio

In [2]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory
from cassandra.query import SimpleStatement
import openai
from llama_index import ListIndex
from llama_index.readers.schema.base import Document
from IPython.display import Markdown, display

# Keys & Environment Variables

In [3]:
# keys and tokens here
openai_api_key = ""
openai.api_key = openai_api_key
cass_user = ''
cass_pw = ''
scb_path = 'secure-connect-vector-search-demo.zip'

# Select a model to compute embeddings

In [4]:
model_id = "text-embedding-ada-002"

# Connect to the Cluster

In [ ]:
cloud_config= {
  'secure_connect_bundle': scb_path
}
auth_provider = PlainTextAuthProvider(cass_user, cass_pw)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()
session.set_keyspace('workspan')

In [6]:
def get_queries_across_opportunities():
    # This function returns a list of queries.
    # You can modify this to fetch the options from a different source if required.
    queries = ['Identify the wins',
               'Identify opportunities with next step to schedule a meeting',
               'I want to know more about the customer and the challenges']
    return queries

In [7]:
def get_queries_specific_to_opportunity():
    # This function returns a list of queries.
    # You can modify this to fetch the options from a different source if required.
    queries = ['What is the sentiment on this opportunity?',
               'What are the next steps for this opportunity?',
               'What are the open items for this opportunity?',
               'What are the challenges for this opportunity?']
    return queries

# Opportunity Specific Queries

In [8]:
def query_opportunity_sentiment(opportunity):
    # This function returns a list of opportunities.
    # You can modify this to fetch the options from a different source if required.
    cqlSelect = f'''SELECT * FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' and opportunity_id = '{opportunity}' ;'''
    rows = session.execute(cqlSelect)
    results = []
    for row_i, row in enumerate(rows):
        results.append(row.sentiment)
    return results[0]

In [9]:
import re

def query_opportunity_nextsteps(opportunity):
    # This function returns a list of opportunities.
    # You can modify this to fetch the options from a different source if required.
    cqlSelect = f'''SELECT * FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' and opportunity_id = '{opportunity}' ;'''
    rows = session.execute(cqlSelect)
    results = []
    for row_i, row in enumerate(rows):
        results.append(row.llm_output)

    next_steps = ["Next Steps section not found!"]
    # Find the section for "Next Steps"
    match = re.search(r'Next Steps:(.*?)(?=Challenges:|Open Items:|$)', results[0], re.DOTALL | re.IGNORECASE)

    if match:
        next_steps_section = match.group(1).strip()
        next_steps = [step.strip('- ').strip() for step in next_steps_section.split('\n') if step]

    next_steps_str = " ".join([step if step.endswith('.') else step + '.' for step in next_steps])

    return next_steps_str


In [10]:
def query_opportunity_openitems(opportunity):
    # This function returns a list of opportunities.
    # You can modify this to fetch the options from a different source if required.
    cqlSelect = f'''SELECT * FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' and opportunity_id = '{opportunity}' ;'''
    rows = session.execute(cqlSelect)
    results = []
    for row_i, row in enumerate(rows):
        results.append(row.llm_output)

    open_items = ["Open Items section not found!"]
    # Find the section for "Open Items"
    match = re.search(r'Open Items:(.*?)(?=Challenges:|Next Steps:|$)', results[0], re.DOTALL | re.IGNORECASE)

    if match:
        open_items_section = match.group(1).strip()
        open_items = [item.strip('- ').strip() for item in open_items_section.split('\n') if item]

    open_items_str = " ".join([item if item.endswith('.') else item + '.' for item in open_items])

    return open_items_str

In [11]:
def query_opportunity_challenges(opportunity):
    # This function returns a list of opportunities.
    # You can modify this to fetch the options from a different source if required.
    cqlSelect = f'''SELECT * FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' and opportunity_id = '{opportunity}' ;'''
    rows = session.execute(cqlSelect)
    results = []
    for row_i, row in enumerate(rows):
        results.append(row.llm_output)

    challenges = ["Challenges section not found!"]
    # Find the section for "Challenges"
    match = re.search(r'Challenges:(.*?)(?=Next Steps:|Open Items:|$)', results[0], re.DOTALL | re.IGNORECASE)

    if match:
        challenges_section = match.group(1).strip()
        challenges = [challenge.strip('- ').strip() for challenge in challenges_section.split('\n') if challenge]

    challenges_str = " ".join([challenge if challenge.endswith('.') else challenge + '.' for challenge in challenges])

    return challenges_str

In [12]:
def get_opportunities():
    # This function returns a list of oppertunities.
    # You can modify this to fetch the options from a different source if required.
    cqlSelect = f'''SELECT opportunity_id FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS'  ;'''
    rows = session.execute(cqlSelect)
    opportunities = []
    for row_i, row in enumerate(rows):
        opportunities.append(row.opportunity_id)
    return opportunities

# Customer Specific Queries (across multiple opportunities)

In [13]:
def query_customer_wins():
    # This function returns a list of opportunities.
    # You can modify this to fetch the options from a different source if required.
    cqlSelect = f'''SELECT * FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' and sentiment = 'positive'  ;'''
    rows = session.execute(cqlSelect)
    results = []
    for row_i, row in enumerate(rows):
        results.append(row.opportunity_id)

    results_str = " ".join([result if result.endswith('.') else result + '.' for result in results])

    return results_str

In [14]:
def query_customer_next_step_to_schedule_meeting():
    # This function returns a list of opportunities.
    # You can modify this to fetch the options from a different source if required.
    vectorsearchon = 'next action to set up a meeting'
    embedding = openai.Embedding.create(input= vectorsearchon, model=model_id)['data'][0]['embedding']
    cqlSelect = f'''SELECT * FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' and llm_output : 'schedule a meeting' ORDER BY llm_output_embedding ANN OF {embedding} LIMIT 10;'''
    rows = session.execute(cqlSelect)
    results = []
    for row_i, row in enumerate(rows):
        next_steps = ["Schedule a Meeting section not found!"]
        # Find the section for "Next Steps"
        match = re.search(r'Schedule a Meeting:(.*?)(?=Challenges:|Open Items:|$)', row.llm_output, re.DOTALL | re.IGNORECASE)

        if match:
            next_steps_section = match.group(1).strip()
            next_steps = [step.strip('- ').strip() for step in next_steps_section.split('\n') if step]

        next_steps_str = " ".join([step if step.endswith('.') else step + '.' for step in next_steps])
        results.append(row.opportunity_id + ' : ' + next_steps_str + '\n' + '\n')

    results_str = " ".join([result if result.endswith('.') else result + '.' for result in results])

    return results_str

In [15]:
def query_customer_challenges():
    vectorsearchon = 'find opportunity with listed challenges'
    embedding = openai.Embedding.create(input= vectorsearchon, model=model_id)['data'][0]['embedding']

    cqlSelect = f'''SELECT llm_output FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' ORDER BY llm_output_embedding ANN OF {embedding} LIMIT 10;  '''
    rows = session.execute(cqlSelect)
    #print(rows)
    documents = []
    for item in rows:
        documents.append(Document(text=str(item)))
        #print(str(item))

    index = ListIndex.from_documents(documents)

    # set Logging to DEBUG for more detailed outputs
    query_engine = index.as_query_engine()
    response = query_engine.query("What are the Challenges?")

    return f"{response}"

In [20]:
def query_customer_llmoutput():
    cqlSelect = f'''SELECT llm_output FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' ;  '''
    rows = session.execute(cqlSelect)
    llmoutputs = []

    for item in rows:
        llmoutputs.append(Document(text=str(item)))

    return llmoutputs

In [17]:
# Function to be called on button click
def on_oppo_button_click(opportunity,query):
    if query == "What is the sentiment on this opportunity?":
        result = query_opportunity_sentiment(opportunity)
    elif query == "What are the next steps for this opportunity?":
        result = query_opportunity_nextsteps(opportunity)
    elif query == "What are the open items for this opportunity?":
        result = query_opportunity_openitems(opportunity)
    elif query == "What are the challenges for this opportunity?":
        result = query_opportunity_challenges(opportunity)
    return result

In [18]:
# Function to be called on button click
def on_cus_button_click(query):
    if query == "Identify the wins":
        result = query_customer_wins()
    elif query == "Identify opportunities with next step to schedule a meeting":
        result = query_customer_next_step_to_schedule_meeting()
    elif query == "I want to know more about the customer and the challenges":
        result = query_customer_challenges()
    return result

# UI Demo - Pre-determined queries

In [44]:
import gradio as gr
from PIL import Image

# Logo and title using the Markdown component
logo_markdown = "![Company Logo]"

with gr.Blocks(theme=gr.themes.Glass(),css=".gradio-container {background-color: lightgrey}") as demo:
    gr.Markdown(logo_markdown)  # Adding the logo and title
    # List of items for the dropdown
    dropdown_opportunity = gr.inputs.Dropdown(choices=get_opportunities(), label="Select an opportunity:")
    dropdown_query = gr.inputs.Dropdown(choices=get_queries_specific_to_opportunity(), label="Select Query:")
    output = gr.Textbox(label="Output Box")
    submit_btn = gr.Button("Submit")
    submit_btn.click(fn=on_oppo_button_click, inputs=[dropdown_opportunity, dropdown_query], outputs=output, api_name="on_oppo_button_click")

    dropdown_cus_query = gr.inputs.Dropdown(choices=get_queries_across_opportunities(), label="Select Query:")
    output_cus = gr.Textbox(label="Output Box")
    submit_cus_btn = gr.Button("Submit")
    submit_cus_btn.click(fn=on_cus_button_click, inputs=dropdown_cus_query, outputs=output_cus, api_name="on_cus_button_click")

demo.launch()


<ipython-input-44-3bd8f76ea77e>:10: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  dropdown_opportunity = gr.inputs.Dropdown(choices=get_opportunities(), label="Select an opportunity:")
<ipython-input-44-3bd8f76ea77e>:10: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  dropdown_opportunity = gr.inputs.Dropdown(choices=get_opportunities(), label="Select an opportunity:")
<ipython-input-44-3bd8f76ea77e>:11: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  dropdown_query = gr.inputs.Dropdown(choices=get_queries_specific_to_opportunity(), label="Select Query:")
<ipython-input-44-3bd8f76ea77e>:11: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  dropdown_query = gr.inputs.Dropdown(choices=get_queries_


Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f315719ee0e735e9b1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# UI Demo - ChatBot with LangChain (with custom queries)

In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import openai
import gradio as gr
import os

os.environ["OPENAI_API_KEY"] = openai_api_key  # Replace with your key

llm = ChatOpenAI(temperature=1.0, model='gpt-3.5-turbo-0613')

# Load custom data
custom_data = query_customer_llmoutput()

def predict(message, history):
    # Checking if the message is in custom data
    if message == "What are the customer challenges?" :
        index = ListIndex.from_documents(custom_data)
        query_engine = index.as_query_engine()
        result = query_engine.query("What are the Challenges?")
        response = result.response
    else:
        history_langchain_format = []
        for human, ai in history:
            history_langchain_format.append(HumanMessage(content=human))
            history_langchain_format.append(AIMessage(content=ai))
        history_langchain_format.append(HumanMessage(content=message))
        gpt_response = llm(history_langchain_format)
        response = gpt_response.content
    return response

gr.ChatInterface(predict).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://8491d55a04a8218e8f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# UI Demo - ChatBot directly integrating to OpenAI

In [46]:
import openai
import gradio as gr

openai.api_key = openai_api_key  # Replace with your key

#llm = ChatOpenAI(temperature=1.0, model='gpt-3.5-turbo-0613')

def predict(message, history):
    history_openai_format = []
    for human, assistant in history:
        history_openai_format.append({"role": "user", "content": human })
        history_openai_format.append({"role": "assistant", "content":assistant})
    history_openai_format.append({"role": "user", "content": message})

    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages= history_openai_format,
        temperature=1.0,
        stream=True
    )

    partial_message = ""
    for chunk in response:
        if len(chunk['choices'][0]['delta']) != 0:
            partial_message = partial_message + chunk['choices'][0]['delta']['content']
            yield partial_message

gr.ChatInterface(predict).queue().launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://eab7254e333d80f57c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
